In [14]:
!pip install audiomentations

In [15]:
#This notebook have been made on google colab. In a local environment, other library might have to be installed
import cv2
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from tqdm import tqdm
import ast
import pandas as pd
import tensorflow as tf
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
import random
import numpy as np
from skimage.transform import resize
from keras.callbacks import EarlyStopping

In [16]:
#Transforms an audio file into a mel spectrogram
def generate_mel_spectrogram(audio, sr):

    S = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels= 128, fmax= 8000,n_fft = 2048 , hop_length = 512 )

    S_dB = librosa.power_to_db(S, ref=np.max)

    return S_dB


#resize a spectrogram
def resize_melspectrogram(melspectrogram):
    resized_mel = resize(
        melspectrogram,
        (128, 128),
        mode='constant',
        anti_aliasing=True
    )
    return resized_mel

In [17]:
df = pd.read_csv("development.csv")#path to dataset
df["tempo"] = df["tempo"].apply(ast.literal_eval)
df["tempo"] = df["tempo"].apply(lambda x: x[0])
df = pd.get_dummies(df, columns=['gender'], prefix='gender',dtype='int')


import tha data

In [ ]:
images = []
labels = []
data = []
for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Traitement des lignes", unit="ligne"):


  file_path = f"{row['path']}"#path to audios
  age = row['age']
  audio, sr = librosa.load(file_path)
  mel_spec = generate_mel_spectrogram(audio,sr)
  mel_spec = resize_melspectrogram(mel_spec)
  #print(f"Forme avant ajout de la dimension de couleur: {mel_spec.shape}")
  # Convertir en tableau d'images
  mel_spec = np.expand_dims(mel_spec, axis=-1)  # Ajouter la dimension de couleur (1 canal)
  #print(f"Forme après ajout de la dimension de couleur: {mel_spec.shape}")
  mel_spec = img_to_array(mel_spec)  # Convertir en format utilisable par Keras
  images.append(mel_spec)
  labels.append(age)  # Assigner l'étiquette de la classe
  vector = row[[
    #'sampling_rate',
    'gender_male',
    'gender_female',
    #'ethnicity',
    'mean_pitch',
    'max_pitch',
    'min_pitch',
    'jitter',
    'shimmer',
    'energy',
    'zcr_mean',
    'spectral_centroid_mean',
    'tempo',
    'hnr',
    'num_words',
    'num_characters',
    'num_pauses',
    'silence_duration',
    #'path'
    ]]

  data.append(vector.to_numpy())

standization

In [20]:
X_images = np.array(images)
X_data = np.array(data)
y = np.array(labels)

X_data = [(X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0)) for X in X_data]
X_images = [(X - X.min()) / (X.max() - X.min()) for X in X_images]

X_images_train, X_images_test, X_data_train, X_data_test, y_train, y_test = train_test_split(
    X_images, X_data, y, test_size=0.2, random_state=42
)

X_images_test = np.array(X_images_test, dtype=np.float32)
X_data_test = np.array(X_data_test, dtype=np.float32)

X_images_train = np.array(X_images_train, dtype=np.float32)
X_data_train = np.array(X_data_train, dtype=np.float32)

logarithm transformation

In [ ]:
#I put it un comment because it makes the results less readable
"""y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)"""

CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, concatenate,Multiply
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

input_img = Input(shape=(128, 128, 1))
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)

#input data
input_data = Input(shape=(X_data_train.shape[1],))
data_transformed  = Dense(64, activation='relu')(input_data)
data_transformed = Dense(32, activation='relu')(data_transformed )
merged = concatenate([x, data_transformed ])

merged = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(merged)
merged = Dense(32, activation='relu')(merged)
output = Dense(1, activation='linear')(merged)  #linear activation for regression

model_final = Model(inputs=[input_img, input_data], outputs=output)
model_final.compile(optimizer=Adam(), loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])
model_final.summary()


model_final.fit([X_images_train, X_data_train], y_train, epochs=25, batch_size=32,validation_data = ([X_images_test, X_data_test], y_test))

evaluation

In [ ]:

test_loss_eval, test_rmse_eval = model_final.evaluate([X_images_test, X_data_test], y_test)
print(f'Loss: {test_loss_eval}, RMSE: {test_rmse_eval}')